In [1]:
%load_ext autoreload
%autoreload 2

import sys
path = '/app'
sys.path.insert(1, path)

from pathlib import Path
import pickle
from tqdm import tqdm
import numpy as np
from face_lib.datasets import IJBDataset, IJBATest, IJBCTemplates
from face_lib.evaluation.aggregation import aggregate_templates

In [32]:
K = 3
D = 2
n = 2
num_z_samples = 1
z = np.array(
    [[1, 2],
     [3, 4],
    ]
)
sigma = np.array(
    [
    [3, 5],
    [2, 1],
    [4, 3]
    ]
)
mu = np.array(
    [[-2, 1],
     [1, 1],
     [2, 2]]
)
z = z[:,np.newaxis,:]

In [33]:
from face_lib.probability.likelihoods import default_pfe_likelihood, default_pfe_likelihood_slow

a_1 = default_pfe_likelihood(mu, sigma, z)* -2
a_2 = default_pfe_likelihood_slow(mu, sigma, z) * -2

100%|██████████| 2/2 [00:00<00:00, 9974.56it/s]


In [34]:
a_1[:,0,:]

array([[ 5.9080502 ,  1.69314718,  2.73490665],
       [12.84138353, 11.69314718,  4.06823998]])

In [35]:
a_2[:,0,:]

array([[ 5.9080502 ,  1.69314718,  2.73490665],
       [12.84138353, 11.69314718,  4.06823998]])

In [53]:
z = np.random.rand(19467, 1, 10)
mu = np.random.rand(3497, 10)
sigma = np.random.rand(3497, 10)

a_1 = default_pfe_likelihood(mu, sigma, z)* -2
a_2 = default_pfe_likelihood_slow(mu, sigma, z) * -2

np.allclose(a_1, a_2)

100%|██████████| 10/10 [00:18<00:00,  1.81s/it]


True

In [55]:
cache_dir = '/app/cache'
uncertainty_strategy = 'head'
dataset_path = "/app/data/big"
protocol_path = "/app/data/test1"

testset = IJBDataset(dataset_path)
image_paths = testset["abspath"].values

features_path = Path(cache_dir) / f"{uncertainty_strategy}_features.pickle"
uncertainty_path = (
    Path(cache_dir) / f"{uncertainty_strategy}_uncertainty.pickle"
)

with open(features_path, "rb") as f:
    feature_dict = pickle.load(f)
with open(uncertainty_path, "rb") as f:
    uncertainty_dict = pickle.load(f)


In [56]:
tester = IJBCTemplates(image_paths, feature_dict, uncertainty_dict)
tester.init_proto(protocol_path)

aggregate_templates(tester.enroll_templates(), 'PFE')
aggregate_templates(tester.verification_templates(), "first")



Cosine distance:
\begin{equation*}
    a^{ijl} = \sum_{k=1}^d\left[(z^{il}_k\cdot\hat{\mu}^j_k)\right]
\end{equation*}
Default PFE:
\begin{equation*}
    a^{ijl} = -\frac{1}{2}\sum_{k=1}^d\left[(z^{il}_k-\hat{\mu}^j_k)^2\frac{1}{{\hat{\sigma}^j_k}^2} +\log{\hat{\sigma}^j_k}^2\right]
\end{equation*}

\begin{equation*}
    a^{ijl} = -\frac{1}{2}\sum_{k=1}^d\left[\left(\frac{z^{il}_k}{\hat{\sigma}^j_k} -\frac{\hat{\mu}^j_k}{\hat{\sigma}^j_k}\right)^2 +\log{\hat{\sigma}^j_k}^2\right]
\end{equation*}

Cosine PFE:
\begin{equation*}
    a^{ijl} = \frac{1}{2}\sum_{k=1}^d\left[(z^{il}_k\cdot\hat{\mu}^j_k)\frac{1}{{\hat{\sigma}^j_k}^2} \cancel{-\log{\hat{\sigma}^j_k}^2}\right]
\end{equation*}



In [4]:
from multiprocessing import Pool
class Compute_scores:
    def __init__(self, mus) -> None:
        self.mus = mus
    def __call__(self, data_piece):
        mu, sigma_sq = data_piece
        enroll_template_sigma_sqrt = np.sqrt(sigma_sq)
        log_sigma_sum = np.log(sigma_sq).sum()
        z_dist_to_mu = []
        z_dist_to_mu_over_sigma = []
        for mu_ver in self.mus:
            z_dist_to_mu.append(np.linalg.norm(mu_ver - mu)**2)
            z_dist_to_mu_over_sigma.append(np.linalg.norm((mu_ver - mu)/enroll_template_sigma_sqrt)**2)
        return log_sigma_sum, z_dist_to_mu, z_dist_to_mu_over_sigma

enroll_data = [(t.mu, t.sigma_sq) for t in tester.enroll_templates()]
mus = [t.mu for t in tester.verification_templates()]
compute_scores = Compute_scores(mus)
with Pool(20) as pool_fc:
    data = pool_fc.map(compute_scores, enroll_data)

In [8]:
z_dist_to_mu = []
log_sigma_sum = []
z_dist_to_mu_over_sigma = []

for t in data:
    log_sigma_sum.append(t[0])
    z_dist_to_mu += t[1]
    z_dist_to_mu_over_sigma+=t[2]

In [21]:
sigma_sq = []
for t in enroll_data:
    sigma_sq.append(t[1])

In [22]:
z_dist_to_mu = np.array(z_dist_to_mu)
log_sigma_sum = np.array(log_sigma_sum)
z_dist_to_mu_over_sigma = np.array(z_dist_to_mu_over_sigma)
sigma_sq = np.array(sigma_sq)

In [16]:
z_dist_to_mu.mean(), z_dist_to_mu.std()

(1.9950265130518914, 0.1360088922879119)

In [19]:
log_sigma_sum.mean(), log_sigma_sum.std()

(-3727.7708, 192.98286)

In [20]:
z_dist_to_mu_over_sigma.mean(), z_dist_to_mu_over_sigma.std()

(3094.800804568953, 1107.134084335944)

In [23]:
sigma_sq.mean(), sigma_sq.std()

(0.00074784114, 0.0003429491)

In [57]:
z = []  # n x num_z_samples x 512
sigma = []  # K x 512
mu = []  # K x 512

# sample z's for each query image
for query_template in tqdm(tester.verification_templates()):
    z_samples = []
    z_samples.append(query_template.mu)
    z.append(z_samples)
z = np.array(z)
for enroll_template in tester.enroll_templates():
    sigma.append(enroll_template.sigma_sq)
    mu.append(enroll_template.mu)
sigma = np.array(sigma)
mu = np.array(mu)

100%|██████████| 19467/19467 [00:00<00:00, 411694.34it/s]


In [112]:
sigma = sigma / (np.min(sigma))

In [113]:
# norm_sigma
np.min(sigma)

1.0

In [114]:
sigma.mean()

4.948837

In [115]:
np.max(sigma)

29.736185

In [116]:
z_range = [22, 44,566,444, 19466]
a_1 = default_pfe_likelihood(mu, sigma, z[z_range])* -2
a_2 = default_pfe_likelihood_slow(mu, sigma, z[z_range]) * -2

np.allclose(a_1, a_2)

100%|██████████| 512/512 [00:02<00:00, 203.95it/s]


True

In [117]:
a_1.mean()

777.0017

In [118]:
diff = np.abs(a_1 - a_2)[:,0,:]

In [119]:
np.unravel_index(diff.argmax(), diff.shape)

(2, 853)

In [120]:
diff.max()

0.0002619028091430664